<a href="https://colab.research.google.com/github/donaldocelaj/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import pandas as pd
import numpy as np



In [0]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data')


In [3]:
df.head(10)


,b,30.83,0,u,g,w,v,1.25,t,t.1,01,f,g.1,00202,0.1,+
0,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,00043,560,+
1,a,24.50,0.500,u,g,q,h,1.500,t,f,0,f,g,00280,824,+
2,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,00100,3,+
3,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,00120,0,+
4,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,00360,0,+
5,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,00164,31285,+
6,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,00080,1349,+
7,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,00180,314,+
8,b,42.50,4.915,y,p,w,v,3.165,t,f,0,t,g,00052,1442,+
9,b,22.08,0.830,u,g,c,h,2.165,f,f,0,t,g,00128,0,+


In [4]:
# Check how many '?' are in each column with an object datatype
for col in df:
  if df[col].dtypes == object:
    print(df[col][df[col] == '?'].value_counts())


?    12
Name: b, dtype: int64
?    12
Name: 30.83, dtype: int64
?    6
Name: u, dtype: int64
?    6
Name: g, dtype: int64
?    9
Name: w, dtype: int64
?    9
Name: v, dtype: int64
Series([], Name: t, dtype: int64)
Series([], Name: t.1, dtype: int64)
Series([], Name: f, dtype: int64)
Series([], Name: g.1, dtype: int64)
?    13
Name: 00202, dtype: int64
Series([], Name: +, dtype: int64)


In [0]:
df.replace('?', np.NaN, inplace=True)

In [6]:
df.dropna(inplace=True)
df.isnull()

,b,30.83,0,u,g,w,v,1.25,t,t.1,01,f,g.1,00202,0.1,+
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [7]:
df.dtypes

b         object
30.83     object
0        float64
u         object
g         object
w         object
v         object
1.25     float64
t         object
t.1       object
01         int64
f         object
g.1       object
00202     object
0.1        int64
+         object
dtype: object

In [0]:
#df['30.83'] = df['30.83'].astype('int64', errors='ignore')
#df['00202'] = df['00202'].astype('int64', errors='ignore')


In [0]:
df['30.83'] = pd.to_numeric(df['30.83'],downcast='float')

In [0]:
df['0'] = pd.to_numeric(df['0'],downcast='float')

In [0]:
df['01'] = pd.to_numeric(df['01'],downcast='float')

In [0]:
df['0.1'] = pd.to_numeric(df['0.1'],downcast='float')

In [0]:
df['00202'] = pd.to_numeric(df['00202'],downcast='float')

In [13]:
df.dtypes

b         object
30.83    float32
0        float32
u         object
g         object
w         object
v         object
1.25     float64
t         object
t.1       object
01       float32
f         object
g.1       object
00202    float32
0.1      float32
+         object
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
import scipy as s

In [15]:
df.head(20)

,b,30.83,0,u,g,w,v,1.25,t,t.1,01,f,g.1,00202,0.1,+
0,a,58.669998,4.460,u,g,q,h,3.040,t,t,6.0,f,g,43.0,560.0,+
1,a,24.500000,0.500,u,g,q,h,1.500,t,f,0.0,f,g,280.0,824.0,+
2,b,27.830000,1.540,u,g,w,v,3.750,t,t,5.0,t,g,100.0,3.0,+
3,b,20.170000,5.625,u,g,w,v,1.710,t,f,0.0,f,s,120.0,0.0,+
4,b,32.080002,4.000,u,g,m,v,2.500,t,f,0.0,t,g,360.0,0.0,+
5,b,33.169998,1.040,u,g,r,h,6.500,t,f,0.0,t,g,164.0,31285.0,+
6,a,22.920000,11.585,u,g,cc,v,0.040,t,f,0.0,f,g,80.0,1349.0,+
7,b,54.419998,0.500,y,p,k,h,3.960,t,f,0.0,f,g,180.0,314.0,+
8,b,42.500000,4.915,y,p,w,v,3.165,t,f,0.0,t,g,52.0,1442.0,+
9,b,22.080000,0.830,u,g,c,h,2.165,f,f,0.0,t,g,128.0,0.0,+


In [16]:
df.describe(exclude=[np.number])

,b,u,g,w,v,t,t.1,f,g.1,+
count,652,652,652,652,652,652,652,652,652,652
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,449,498,498,133,380,348,366,350,597,357


In [17]:
cols = ['30.83', '0', '1.25', '01', '00202', '0.1']
cols

['30.83', '0', '1.25', '01', '00202', '0.1']

In [0]:
chart = pd.DataFrame()

for i in cols:
  plus_class = df[i][df['+'] == '+']
  minus_class = df[i][df['+'] =='-']
  
  t_test= s.stats.ttest_ind(plus_class, minus_class, equal_var=False)
  
  chart.loc[i, 't_test'] = t_test.statistic
  chart.loc[i, 'p_value'] = t_test.pvalue




For this chart, negative T-test values are in favor of the minus class, and positive values are in favor of the plus class.

In [19]:
chart

,t_test,p_value
30.83,4.604541,5.089406e-06
0,5.337353,1.373670e-07
1.25,8.504899,3.496031e-16
01,10.494658,1.676514e-22
00202,-2.199736,2.818343e-02
0.1,4.081645,5.751782e-05


In [0]:
from scipy.stats import chisquare


In [0]:
ct1 = pd.crosstab(df['+'], df['b'])
ct2 = pd.crosstab(df['+'], df['u'])
ct3 = pd.crosstab(df['+'], df['g'])
ct4 = pd.crosstab(df['+'], df['w'])
ct5 = pd.crosstab(df['+'], df['v'])
ct6 = pd.crosstab(df['+'], df['t'])
ct7 = pd.crosstab(df['+'], df['t.1'])
ct8 = pd.crosstab(df['+'], df['f'])
ct9 = pd.crosstab(df['+'], df['g.1'])

In [22]:
ct1

b,a,b
+,,
+,95,200
-,108,249


In [23]:
from scipy.stats import chisquare

chisquare(ct1, axis=None)

Power_divergenceResult(statistic=100.69938650306747, pvalue=1.0992838132115145e-21)

In [24]:
chisquare(ct2, axis=None)

Power_divergenceResult(statistic=613.1717791411043, pvalue=2.882405398828689e-130)

In [25]:
chisquare(ct3, axis=None)

Power_divergenceResult(statistic=613.1717791411041, pvalue=2.8824053988288534e-130)

In [26]:
chisquare(ct4, axis=None)

Power_divergenceResult(statistic=376.7852760736197, pvalue=2.611711085733085e-63)

In [27]:
chisquare(ct5, axis=None)

Power_divergenceResult(statistic=1764.693251533742, pvalue=0.0)

In [28]:
chisquare(ct6, axis=None)

Power_divergenceResult(statistic=353.4601226993865, pvalue=2.657316994756324e-76)

In [29]:
chisquare(ct7, axis=None)

Power_divergenceResult(statistic=151.91411042944785, pvalue=1.0182168668455379e-32)

In [30]:
chisquare(ct8, axis=None)

Power_divergenceResult(statistic=11.644171779141107, pvalue=0.008707001566607924)

In [31]:
chisquare(ct9, axis=None)

Power_divergenceResult(statistic=1010.2944785276072, pvalue=3.54912408254312e-216)

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

I ran a T test for every column and saw that most of the columns reject the null hypothesis.

I ran a chi squared test for each of the 9 non numeric columns, I found out that 8 of the 9 categories significantly reject the null hypothesis and therefore are not very dependent on each other.

I found the first section the most difficult.